## GDAL with HDF5 and VRT

In [ ]:
import numpy as np
from common import earthaccess_args
from osgeo import gdal

In [ ]:
gdal.UseExceptions()

In [ ]:
dataset = "gpm_imerg"
filename = earthaccess_args[dataset]["filename"]
variable = earthaccess_args[dataset]["variable"]
scale = earthaccess_args[dataset]["scale"]
a_ullr = earthaccess_args[dataset]["a_ullr"]

In [ ]:
def load_data():
    src = f"vrt://earthaccess_data/{filename}?a_ullr={a_ullr}&sd_name={variable}"
    ds = gdal.Open(src)
    band = ds.GetRasterBand(1)
    arr = band.ReadAsArray().astype("float32", casting="unsafe")
    mask = band.GetMaskBand().ReadAsArray()
    ma = np.ma.masked_array(arr, np.logical_not(mask), fill_value=np.nan)
    if scale:
        np.multiply(ma, band.GetScale(), out=ma, casting="unsafe")
        np.add(ma, band.GetOffset(), out=ma, casting="unsafe")
    return ma.filled(fill_value=np.nan)

In [ ]:
if __name__ == "__main__":
    da = load_data()